# Solar components

* File name: solar_components.ipynb
* Last edited: 2020-06-25
* Created by: Stefan Bruche (TU Berlin)

```python
import pandas as pd
import aristopy as ar


```

<div class="alert alert-info">

**Note:** 
    
An example model to show the functionality of the solar classes is added on this page soon.
</div>